In [ ]:
import pickle

In [ ]:
with open('../../FedLGR_SocRob/models/mod0.pkl', 'rb') as f:
    weights = pickle.load(f)

In [ ]:


import os
from PIL import Image

# Set this to your images directory
folder_path = 'Rafal_MSc_Thesis/IMAGES/'  # Change this if needed

for filename in os.listdir(folder_path):
    if filename.lower().endswith('.png'):
        file_path = os.path.join(folder_path, filename)
        try:
            with Image.open(file_path) as img:
                width, height = img.size
            file_size_kb = os.path.getsize(file_path) / 1024
            print(f"{width}x{height} pixels, {file_size_kb:.2f} KB")
        except Exception as e:
            print(f"Error processing {filename}: {e}")


In [ ]:
from PIL import Image
import os

# Set this to your images folder (or '.' for current directory)
folder_path = 'Rafal_MSc_Thesis/IMAGES/'
size_threshold_kb = 3000  # 3 MB
resize_factor = 0.8  # Resize to 50% (half width, half height)

for filename in os.listdir(folder_path):
    # Check for PNG, JPG, JPEG files (case-insensitive)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(folder_path, filename)
        try:
            file_size_kb = os.path.getsize(file_path) / 1024
            if file_size_kb > size_threshold_kb:
                with Image.open(file_path) as img:
                    width, height = img.size
                    new_size = (int(width * resize_factor), int(height * resize_factor))
                    img_resized = img.resize(new_size, Image.LANCZOS)  # High quality
                    output_path = os.path.join('Rafal_MSc_Thesis/newimg' ,filename)
                    img_resized.save(output_path)
                    print(f"Resized {filename} ({file_size_kb:.2f} KB) to {new_size[0]}x{new_size[1]} and saved as {output_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")


In [ ]:
print(f"Loaded weights type: {type(weights)}")

if isinstance(weights, list):
    print(f"Number of weight arrays saved: {len(weights)}")
    # Optionally print shape of each weight array
    for i, arr in enumerate(weights):
        print(f"Weight[{i}] shape: {arr.shape}")
elif isinstance(weights, dict):
    print(f"Dict keys: {list(weights.keys())}")
else:
    print("Saved weights are neither a list nor dict, raw object:")
    print(weights)

In [ ]:
import torch

# Paths to the weight files
gpu_weights_path = "../../FedLGR_SocRob/models/gpu/MobileNet.pkl"
cpu_weights_path = "../../FedLGR_SocRob/models/cpu/MobileNet.pkl"

# Load weights mapping both to CPU
weights_gpu = torch.load(gpu_weights_path, map_location=torch.device('cpu'))
weights_cpu = torch.load(cpu_weights_path, map_location=torch.device('cpu'))

# Function to compare two state_dicts
def compare_state_dicts(sd1, sd2):
    if sd1.keys() != sd2.keys():
        print("Different parameter sets")
        return False
    for key in sd1.keys():
        if not torch.equal(sd1[key], sd2[key]):
            print(f"Parameter differs: {key}")
            print(f" - GPU shape: {sd1[key].shape}, CPU shape: {sd2[key].shape}")
            diff = torch.norm(sd1[key] - sd2[key])
            print(f" - Norm of difference: {diff.item()}")
            return False
    print("All parameters match")
    return True

# Assuming the files contain state_dicts (dict mapping param names to tensors)
compare_state_dicts(weights_gpu, weights_cpu)


In [ ]:
import pickle
import torch

def load_pickle_as_state_dict(pkl_path):
    with open(pkl_path, 'rb') as f:
        state_dict = pickle.load(f)

    # Convert all numpy arrays in state_dict to torch tensors
    for key in state_dict:
        if isinstance(state_dict[key], (list, tuple)):
            # Sometimes parameters can be list or tuple
            state_dict[key] = torch.tensor(state_dict[key])
        elif hasattr(state_dict[key], 'shape'):  # numpy ndarray
            state_dict[key] = torch.tensor(state_dict[key])
        else:
            # Could already be tensor or scalar
            pass
    return state_dict

# Example usage
cpu_weights_path = "../../FedLGR_SocRob/models/cpu/MobileNet.pkl"
gpu_weights_path = "../../FedLGR_SocRob/models/gpu/MobileNet.pkl"

cpu_sd = load_pickle_as_state_dict(cpu_weights_path)
gpu_sd = load_pickle_as_state_dict(gpu_weights_path)

def compare_state_dicts(sd1, sd2):
    keys1 = set(sd1.keys())
    keys2 = set(sd2.keys())
    if keys1 != keys2:
        print("Different parameter keys found:")
        print("In sd1 but not in sd2:", keys1 - keys2)
        print("In sd2 but not in sd1:", keys2 - keys1)
        return False

    all_close = True
    for key in sd1.keys():
        tensor1 = sd1[key]
        tensor2 = sd2[key]

        # Move both tensors to CPU before comparison
        if tensor1.device != torch.device('cpu'):
            tensor1 = tensor1.to('cpu')
        if tensor2.device != torch.device('cpu'):
            tensor2 = tensor2.to('cpu')

        if not torch.allclose(tensor1, tensor2, atol=1e-6):
            diff = (tensor1 - tensor2).abs().max()
            print(f"Parameter '{key}' differs, max abs diff = {diff}")
            all_close = False

    if all_close:
        print("All parameters match closely!")
    return all_close


compare_state_dicts(cpu_sd, gpu_sd)
